<a href="https://colab.research.google.com/github/santhoshmohan05/NeuralNetworks/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h4><b>RNN with LSTM <b></h4>

In [0]:
from google.colab import drive
from os import listdir
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dropout,LSTM,Dense

In [0]:
drive.mount('/content/gdrive/')

In [0]:
train_data_file = '/content/gdrive/My Drive/CNN_Test_Datat/Google_Stock_Price_Train.csv'
test_data_file = '/content/gdrive/My Drive/CNN_Test_Datat/Google_Stock_Price_Test.csv'
train_data = pd.read_csv(train_data_file)
test_data = pd.read_csv(test_data_file)
train_data.head()

<h4><b>Data Preprocessing </b></h4>

In [0]:
##processing only the open value in order
training_set = train_data.iloc[:,1:2].values
sc = MinMaxScaler(feature_range=(0,1))
scaled_train_data = sc.fit_transform(training_set)

stock_interval = 60
### create X and Y for each step of the RNN
X_train = []
y_train = []
for i in range(stock_interval,scaled_train_data.shape[0]):
    X_train.append(scaled_train_data[i-stock_interval:i,0])
    y_train.append(scaled_train_data[i,0])
X_train = np.array(X_train)
y_train = np.array(y_train)

### reshaping an numpy array 
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))

<h4><b>RNN Regressor Compilation </b></h4>


In [0]:
def get_RNN_regressor(memory_units,dropout_precentage,X):
    regressor = Sequential()
    ### 4 LSTM Layers with 50 neurons every laer and 20% dropout
    regressor.add(LSTM(units=memory_units,return_sequences=True,input_shape=(X.shape[1],X.shape[2])))
    regressor.add(Dropout(dropout_precentage))
    regressor.add(LSTM(units=memory_units,return_sequences=True))
    regressor.add(Dropout(dropout_precentage))
    regressor.add(LSTM(units=memory_units,return_sequences=True))
    regressor.add(Dropout(dropout_precentage))
    regressor.add(LSTM(units=memory_units))
    regressor.add(Dropout(dropout_precentage))
    ### output layer - fully connected Layer with one output and linear activation.
    regressor.add(Dense(units=1))
    return regressor

regressor = get_RNN_regressor(50,0.2,X_train)
### adam or rmsprop optimizer with MSE loss function
regressor.compile(optimizer='adam', loss='mean_squared_error')

<h4><b>Fit the RNN</b></h4>


In [0]:
regressor.fit(X_train,y_train,batch_size=8,epochs=100)

<h4><b>Prediction</b></h4>

In [0]:
real_stock_price = test_data.iloc[:, 1:2].values

# Getting the predicted stock price of 2017
dataset_total = pd.concat((train_data['Open'], test_data['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(test_data) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)
predicted_stock_price

**Visualizing the results**

In [0]:
# Visualising the results
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()